<h1> Deploying and predicting with model </h1>

This notebook illustrates:
<ol>
<li> Deploying model
<li> Predicting with model
</ol>

In [14]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-1796578b8d4ae7cc'
PROJECT = 'qwiklabs-gcp-1796578b8d4ae7cc'
REGION = 'europe-west1'

In [15]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.7'  

In [16]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

In [17]:
%bash
# copy solution to Lab #5 (skip this step if you still have results from Lab 5 in your bucket)
gsutil -m cp -R gs://cloud-training-demos/babyweight/trained_model gs://${BUCKET}/babyweight/trained_model

Copying gs://cloud-training-demos/babyweight/trained_model/checkpoint...
Copying gs://cloud-training-demos/babyweight/trained_model/eval/events.out.tfevents.1529348264.cmle-training-master-a137ac0fff-0-9q8r4...
Copying gs://cloud-training-demos/babyweight/trained_model/events.out.tfevents.1529347276.cmle-training-master-a137ac0fff-0-9q8r4...
Copying gs://cloud-training-demos/babyweight/trained_model/export/exporter/1529348266/saved_model.pb...
Copying gs://cloud-training-demos/babyweight/trained_model/export/exporter/1529348266/variables/variables.data-00000-of-00001...
Copying gs://cloud-training-demos/babyweight/trained_model/export/exporter/1529348266/variables/variables.index...
Copying gs://cloud-training-demos/babyweight/trained_model/export/exporter/1529348864/saved_model.pb...
Copying gs://cloud-training-demos/babyweight/trained_model/export/exporter/1529348864/variables/variables.data-00000-of-00001...
Copying gs://cloud-training-demos/babyweight/trained_model/export/exporter/

## Task 1

What files are present in the model trained directory (gs://${BUCKET}/babyweight/trained_model)?

Hint (highlight to see): <p style='color:white'>
Run gsutil ls in a bash cell.
Answer: model checkpoints are in the trained model directory and several exported models (model architecture + weights) are in the export/exporter subdirectory
</p>

<h2> Task 2: Deploy trained, exported model </h2>

Uncomment and run the the appropriate gcloud lines ONE-BY-ONE to 
deploy the trained model to act as a REST web service.

Hint (highlight to see): <p style='color:white'>
The very first time, you need only the last two gcloud calls to create the model and the version.
To experiment later, you might need to delete any deployed version, but should not have to recreate the model
</p>

In [18]:
%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model/export/exporter/

gs://qwiklabs-gcp-1796578b8d4ae7cc/babyweight/trained_model/export/exporter/1529348266/
gs://qwiklabs-gcp-1796578b8d4ae7cc/babyweight/trained_model/export/exporter/1529348864/
gs://qwiklabs-gcp-1796578b8d4ae7cc/babyweight/trained_model/export/exporter/1529349460/
gs://qwiklabs-gcp-1796578b8d4ae7cc/babyweight/trained_model/export/exporter/1529350065/
gs://qwiklabs-gcp-1796578b8d4ae7cc/babyweight/trained_model/export/exporter/1529350672/
gs://qwiklabs-gcp-1796578b8d4ae7cc/babyweight/trained_model/export/exporter/1529351274/
gs://qwiklabs-gcp-1796578b8d4ae7cc/babyweight/trained_model/export/exporter/1529351873/
gs://qwiklabs-gcp-1796578b8d4ae7cc/babyweight/trained_model/export/exporter/1529352472/
gs://qwiklabs-gcp-1796578b8d4ae7cc/babyweight/trained_model/export/exporter/1529353074/
gs://qwiklabs-gcp-1796578b8d4ae7cc/babyweight/trained_model/export/exporter/1529353677/
gs://qwiklabs-gcp-1796578b8d4ae7cc/babyweight/trained_model/export/exporter/1529354267/
gs://qwiklabs-gcp-1796578b8d4ae7

In [19]:
%bash
MODEL_NAME="babyweight"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/babyweight/trained_model/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Deleting and deploying babyweight ml_on_gcp from gs://qwiklabs-gcp-1796578b8d4ae7cc/babyweight/trained_model/export/exporter/1529355466/ ... this will take a few minutes


Created ml engine model [projects/qwiklabs-gcp-1796578b8d4ae7cc/models/babyweight].
Creating version (this might take a few minutes)......
..................................................................................................................................................................................................................................................................................................................................................................done.


## Task 3: Write Python code to invoke the deployed model (online prediction)

<p>
Send a JSON request to the endpoint of the service to make it predict a baby's weight. The order of the responses are the order of the instances.

The deployed model requires the input instances to be formatted as follows:
<pre>
{
      'key': 'b1',
      'is_male': 'True',
      'mother_age': 26.0,
      'plurality': 'Single(1)',
      'gestation_weeks': 39
},
</pre>
The key is an arbitrary string.  Allowed values for is_male are True, False and Unknown.
Allowed values for plurality are Single(1), Twins(2), Triplets(3), Multiple(2+)

In [20]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = 'babyweight'
MODEL_VERSION = 'ml_on_gcp'

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
# TODO: complete
    {
      'key': 'b1',
      'is_male': 'True',
      'mother_age': 26.0,
      'plurality': 'Single(1)',
      'gestation_weeks': 39
    },
    {
      'key': 'g1',
      'is_male': 'False',
      'mother_age': 29.0,
      'plurality': 'Single(1)',
      'gestation_weeks': 38
    },
    {
      'key': 'b2',
      'is_male': 'True',
      'mother_age': 26.0,
      'plurality': 'Triplets(3)',
      'gestation_weeks': 39
    },
    {
      'key': 'u1',
      'is_male': 'Unknown',
      'mother_age': 29.0,
      'plurality': 'Multiple(2+)',
      'gestation_weeks': 38
    },
  ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{"predictions": [{"predictions": [7.740230083465576], "key": ["b1"]}, {"predictions": [7.247548580169678], "key": ["g1"]}, {"predictions": [6.182091236114502], "key": ["b2"]}, {"predictions": [6.13692569732666], "key": ["u1"]}]}'


<h2> Task 4: Try out batch prediction </h2>
<p>
Batch prediction is commonly used when you thousands to millions of predictions.
Create a file withe one instance per line and submit using gcloud.

In [21]:
%writefile inputs.json
{"key": "b1", "is_male": "True", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}
{"key": "g1", "is_male": "False", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}

Writing inputs.json


In [22]:
%bash
INPUT=gs://${BUCKET}/babyweight/batchpred/inputs.json
OUTPUT=gs://${BUCKET}/babyweight/batchpred/outputs
gsutil cp inputs.json $INPUT
gsutil -m rm -rf $OUTPUT 
gcloud ml-engine jobs submit prediction babypred_$(date -u +%y%m%d_%H%M%S) \
  --data-format=TEXT --region ${REGION} \
  --input-paths=$INPUT \
  --output-path=$OUTPUT \
  --model=babyweight --version=ml_on_gcp

jobId: babypred_181126_095231
state: QUEUED


Copying file://inputs.json [Content-Type=application/json]...
/ [1 files][  204.0 B/  204.0 B]                                                
Operation completed over 1 objects/204.0 B.                                      
CommandException: 1 files/objects could not be removed.
Job [babypred_181126_095231] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe babypred_181126_095231

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs babypred_181126_095231


Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License